In [3]:
"""
LINKS:

- https://math.stackexchange.com/questions/4952613/is-there-a-version-of-weyls-unitary-trick-for-positive-characteristic
- https://mathoverflow.net/questions/327823/unitary-representations-of-finite-groups-over-finite-fields
- https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group

""";

In [1]:
"""
NOTES:

- to obtain unitary rep'ns of S_n over F_{q^2}, first compute the space of G-invariant symmetric bilinear forms for a given rep'n
- for the matrix U associated to such a form, use GAP's BaseChangeToCanonical to factor U=AA^* where * denotes conjugate transpose
- conjugation is x |--> x^q, the r^th power of Frobenius if q=p^r over F_{q^2}, an order two map
- the unitary rep'ns are then \tilde{\rho}(g) = A^*\rho(g)A^*.inverse()

""";

In [1]:
q = 11

In [46]:
#define Specht module representation over GF(q^2)
spc = SymmetricGroupRepresentation([4,2,1,1], "specht", ring=GF(q^2)); spc

Specht representation of the symmetric group corresponding to [4, 2, 1, 1]

In [45]:
#define the orthogonal representation over GF(q^2) for a prime power q
orth = SymmetricGroupRepresentation([4,2,1,1], "orthogonal", ring=GF(q^2)); orth

Orthogonal representation of the symmetric group corresponding to [4, 2, 1, 1]

In [41]:
#NOTE: rep'n matrix doesn't work due to square roots
#throws TypeError: unable to coerce <class 'sage.symbolic.expression.Expression'>
#FIX: changing line 662 of symmetric_group_representations.py to first convert the element to self._ring before passing it
try:
    orth.representation_matrix_for_simple_transposition(2)
except TypeError as e:
    print(e)

unable to coerce <class 'sage.symbolic.expression.Expression'>


In [7]:
#define a new representation_matrix_for_simple_transposition with the fix beta --> self._ring
def new_representation_matrix_for_simple_transposition(orth,i):
        from copy import copy
        if not (1 <= i < sum(orth._partition)):
            raise TypeError
        Y = orth._yang_baxter_graph
        index_lookup = {b: a for a, b in enumerate(list(Y))}
        digraph = copy(Y._digraph)
        digraph.delete_edges((u, v) for (u, v, (j, beta)) in digraph.edges(sort=True)
                             if j != i)
        M = matrix(orth._ring, digraph.num_verts())
        for g in digraph.connected_components_subgraphs():
            if g.num_verts() == 1:
                [v] = g.vertices(sort=True)
                w = orth._word_dict[v]
                trivial = None
                for j, a in enumerate(w):
                    if a == i and w[j + 1] == i + 1:
                        trivial = True
                        break
                    elif a == i + 1:
                        trivial = False
                        break
                j = index_lookup[v]
                M[j, j] = 1 if trivial is True else -1
            else:
                [(u, v, (j, beta))] = g.edges(sort=True)
                iu = index_lookup[u]
                iv = index_lookup[v]
                M[iu, iu], M[iu, iv], M[iv, iu], M[iv, iv] = \
                    orth._2x2_matrix_entries(orth._ring(beta))
        return M

In [8]:
#compute the new representation matrix with the fix
new_representation_matrix_for_simple_transposition(orth,2)

[       3 6*z2 + 4        0        0]
[6*z2 + 4        4        0        0]
[       0        0        6        0]
[       0        0        0        6]

In [9]:
#define representation_matrix_uncached now using new_representation_matrix_for_simple_transposition
def new_representation_matrix_uncached(orth, permutation):
    m = orth._yang_baxter_graph._digraph.num_verts()
    M = matrix(orth._ring, m, m, 1)
    for i in Permutation(permutation).reduced_word():
        M *= new_representation_matrix_for_simple_transposition(orth,i)
    return M

In [10]:
#instantiate an orthogonal matrix over the finite field GF(q^2)
M_orth = new_representation_matrix_uncached(orth,[1,3,2,5,4]); M_orth

[     4 z2 + 3      0      0]
[z2 + 3      3      0      0]
[     0      0      2      5]
[     0      0      5      5]

In [11]:
#check that the resulting matrix is orthogonal
M_orth*M_orth.transpose() == identity_matrix(M_orth.nrows())

True

In [12]:
sum(orth._partition)

5

In [13]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for i in range(A.nrows())] for j in range(A.nrows())])

In [14]:
#check if the matrix is unitary with respect to the form <x,y> = \sum_i x_i*y_i**q
M_orth*conjugate_pos_char(M_orth).transpose() == identity_matrix(M_orth.nrows())

False

In [15]:
M_orth*conjugate_pos_char(M_orth).transpose()

[       3 6*z2 + 4        0        0]
[  z2 + 3        3        0        0]
[       0        0        1        0]
[       0        0        0        1]

In [16]:
#NOTE: the matrix cannot be both orthogonal and unitary if the conjugation map x |--> x**q does not fix every element over extensions

In [31]:
#group permutations by cycle type
def conj_classes(n):
    G = SymmetricGroup(n)
    P_n = Partitions(n)
    C = P_n.cardinality()
    indices = {lam: C - 1 - i for i, lam in enumerate(P_n)}
    cycles = {}
    for g in G:
        ind = indices[g.cycle_type()]
        if ind in cycles:
            cycles[ind].append(g)
        else:
            cycles[ind] = [g]
    return cycles

In [50]:
#twisted Brauer character by automorphism \sigma: x |--> x^q
#Brauer character is eigenvalues of \rho(g) mapped to roots of unity in \C under bijection and summed
#define a rep'n \rho twisted by a field automorphism \sigma by applying \sigma to each matrix entry
#this gives another representation (check this). can then compute sum of eigenvalues to get twisted character
def twisted_brauer_character(repn,rep_type="orth"):
    la = repn._partition
    n = sum(la)
    character_values = []
    for i, cycles in conj_classes(n).items():
        conj_class_rep = cycles[0]
        #compute eigenvalues of \rho(g) over F_q, map to |C and sum
        if rep_type == "orth":
            rho_g = new_representation_matrix_uncached(repn,conj_class_rep)
        if rep_type == "specht":
            rho_g = repn.representation_matrix(Permutation(conj_class_rep))
        splitting_field.<a> = rho_g.charpoly().splitting_field()
        field_size = splitting_field.order()
        rho_g_conj_extend = matrix(splitting_field,conjugate_pos_char(rho_g))
        eigs_w_mult = [(pair[0],len(pair[1])) for pair in rho_g_conj_extend.eigenvectors_right()]
        #use .log(a) for F.<a> = GF(q^s) to get power k. then map to exp(2*pi*i*k/(q^s-1).
        to_roots_of_unity = [pair[1]*exp(2*pi*I*pair[0].log(a)/(field_size-1)) for pair in eigs_w_mult]
        char_value = sum(to_roots_of_unity).full_simplify()
        character_values.append(char_value)
    return character_values

In [51]:
#need to enumerate conjugacy classes to match standard Brauer character
twisted_brauer_character(orth,rep_type="orth")

[90,
 0,
 0,
 2,
 -6,
 -26*cos(3/7*pi) + 26*cos(2/7*pi) - 26*cos(1/7*pi) + 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -6,
 0,
 0,
 2,
 0,
 0,
 0,
 0]

In [48]:
#compute the Brauer character directly
la = orth._partition
n = sum(la)
base_ring = orth._ring.base_ring()
SGA = SymmetricGroupAlgebra(base_ring, n)
brauer_character = SGA.simple_module(la).brauer_character(); brauer_character

(90, 0, -6, 0, -6, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, -1, 0)

In [21]:
SymmetricGroup(n).character_table()

[ 1 -1  1  1 -1 -1  1]
[ 4 -2  0  1  1  0 -1]
[ 5 -1  1 -1 -1  1  0]
[ 6  0 -2  0  0  0  1]
[ 5  1  1 -1  1 -1  0]
[ 4  2  0  1 -1  0 -1]
[ 1  1  1  1  1  1  1]

In [22]:
conj_class_rep = conj_classes(n)[6][0]; conj_class_rep

(1,5,4,3,2)

In [23]:
M_orth = new_representation_matrix_uncached(orth,conj_class_rep); M_orth

[       3   z2 + 3        0        0]
[2*z2 + 6        1        5        0]
[4*z2 + 5        2        4        5]
[3*z2 + 2        5        3        5]

In [24]:
splitting_field.<a> = M_orth.charpoly().splitting_field(); splitting_field

Finite Field in a of size 7^4

In [25]:
splitting_field.order()

2401

In [26]:
d, L = matrix(splitting_field,M_orth).diagonalization(); [eig.log(a) for eig in d.diagonal()]

[480, 1920, 1440, 960]

In [27]:
matrix(splitting_field,M_orth).eigenvectors_right()[0]

(4*a^2 + 2*a + 1,
 [
 (1, a^3 + 4*a^2, 2*a^3 + 6*a^2 + 3*a + 6, 4*a^3 + 4*a^2 + 6*a + 2)
 ],
 1)

In [28]:
conjugate_pos_char(M_orth)

[       3 5*z2 + 1 3*z2 + 2 4*z2 + 5]
[6*z2 + 4        1        2        5]
[       0        5        4        3]
[       0        0        5        5]